In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import datetime
from glob import glob
import IPython.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import scipy.interpolate

import pydicom

In [ ]:
np.__version__

In [ ]:
needs_restart = False

try:
    import pymedphys
    assert pymedphys.__version__ == '0.17.0'
except:
    !pip install pymedphys==0.17.0
    needs_restart = True

if pydicom.__version__ != '1.2.0':
    !pip install pydicom==1.2.0
    needs_restart = True

if needs_restart:
    raise(ImportError("Please restart kernel to use pydicom 1.2.0 and pymedphys 0.17.0"))

In [ ]:
pymedphys.__version__

In [ ]:
pydicom.__version__

In [ ]:
import pymedphys._wlutz.core
import pymedphys._wlutz.reporting

In [ ]:
edge_lengths = [20, 20]
bb_diameter = 8
penumbra = 2

In [ ]:
data_root = r'\\pdc\OneDrive$\NBCCC Specific Files\Linac Beam Data Record\VersaHD 4299\QA\20191217_monthly\wltest'

In [ ]:
output_csv = os.path.join(data_root, 'results.csv')

In [ ]:
data_record = glob(os.path.join(data_root, 'iView*.xlsx'))[0]
dicom_files = np.array(glob(os.path.join(data_root, '*.dcm')))

In [ ]:
dicom_files

In [ ]:
record = pd.read_excel(data_record, skiprows=4)
timestamps_initial = record['Datetime']
timestamps = timestamps_initial[timestamps_initial.notnull()].values
gantry = record['Gantry'][timestamps_initial.notnull()].values
collimator = record['Col'][timestamps_initial.notnull()].values
turntable = record['TT'][timestamps_initial.notnull()].values
beam = record['Energy'][timestamps_initial.notnull()].values
note = record['Note'][timestamps_initial.notnull()].values

In [ ]:
datasets = np.array([
    pydicom.read_file(dicom_file, force=True)
    for dicom_file in dicom_files
])

In [ ]:
acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

In [ ]:
diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

In [ ]:
assert len(set(acquisition_index)) == len(acquisition_index)
assert len(acquisition_index) == len(acquisition_datetimes)

In [ ]:
acquisition_index

In [ ]:
datasets = datasets[acquisition_index]
dicom_files = dicom_files[acquisition_index]
timestamps = timestamps[timestamp_index]
gantry = gantry[timestamp_index]
collimator = collimator[timestamp_index]
turntable = turntable[timestamp_index]
beam = beam[timestamp_index]
note = note[timestamp_index]

acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

assert np.all(timestamp_index == acquisition_index)

In [ ]:
def get_x_y_img(dataset):
    img = dataset.pixel_array
    img = 1 - img[::-1, :] / 2 ** 16
    
    shape = np.shape(img)
    x = np.arange(-shape[1] / 2, shape[1] / 2) / 4
    y = np.arange(-shape[0] / 2, shape[0] / 2) / 4
    
    return x, y, img

In [ ]:
bb_centres = []
field_centres = []
field_rotations = []

for i, dataset in enumerate(datasets):
    x, y, img = get_x_y_img(dataset)
    
    print(gantry[i], collimator[i], turntable[i])
    
    try:
        bb_centre, field_centre, field_rotation = pymedphys._wlutz.core.find_field_and_bb(
            x, y, img, edge_lengths, bb_diameter)
    except ValueError as e:
        print(e)
        bb_centre = [None, None]
        field_centre = [None, None]
        field_rotation = None
    
    bb_centres.append(bb_centre)
    field_centres.append(field_centre)
    field_rotations.append(field_rotation)

In [ ]:
field_centres = np.array(field_centres)
bb_centres = np.array(bb_centres)

field_centres[field_centres == None] = np.nan
bb_centres[bb_centres == None] = np.nan
field_displacements = field_centres - bb_centres

In [ ]:
tabulated_data = pd.DataFrame(
    index=timestamps,
    data=np.array([
        beam, gantry, collimator, turntable, 
        field_centres[:, 0], field_centres[:, 1],
        bb_centres[:, 0], bb_centres[:, 1],
        field_displacements[:, 0], field_displacements[:, 1],  note
    ]).T,
    columns=[
        'Beam', 'Gantry', 'Collimator', 'Turn Table', 
        'Field Centre x (mm)', 'Field Centre y (mm)',
        'BB Centre x (mm)', 'BB Centre y (mm)',
        'Field - BB x (mm)', 'Field - BB y (mm)', 'Note'
    ]
)

In [ ]:
tabulated_data.to_csv(output_csv)

In [ ]:
string_timestamps = tabulated_data.index.strftime('%Y%m%d_%H%M%S').values

In [ ]:
for i, (dataset, bb_centre, field_centre, field_rotation) in enumerate(zip(datasets, bb_centres, field_centres, field_rotations)):
    x, y, img = get_x_y_img(dataset)
    
    if np.isnan(bb_centre[0]):
        plt.contourf(x,y, img, 100)
        plt.xlim([-20,20])
        plt.ylim([-20,20])
        plt.axis('equal')
        
    else:
        pymedphys._wlutz.reporting.image_analysis_figure(
            x,
            y,
            img,
            bb_centre,
            field_centre,
            field_rotation,
            bb_diameter,
            edge_lengths,
            penumbra,
        )

        file_name = "{}_{}_G{:+04.0f}_C{:+04.0f}_TT{:+03.0f}".format(
            string_timestamps[i], beam[i], gantry[i], collimator[i], turntable[i])

        file_path = os.path.join(data_root, file_name)

        plt.savefig(file_path)    

    plt.show()